In [51]:
import pandas as pd
import pulp as lp
import numpy as np
from operator import iadd
from functools import reduce
from typing import Sequence, Any, DefaultDict, List

In [52]:
def create_prob(prob_name: str, sense: int) -> lp.LpProblem:
    return lp.LpProblem(prob_name, sense)


def add_obj_fn(lp_prob: lp.LpProblem, dvar: lp.LpAffineExpression) -> lp.LpProblem:
    return iadd(lp_prob, dvar)


def add_constraint(lp_prob: lp.LpProblem, constrs: Sequence[lp.LpConstraint]) -> lp.LpProblem:
   return reduce(iadd, constrs, lp_prob)


def head(x: Sequence) -> Any:
    return x[0]

def to_str(indnum, activity) -> str:
    return f'{indnum} - {activity}'

In [53]:
df = pd.read_csv('data/dataset.csv')

df.drop('Unnamed: 0', axis=1, inplace=True) # Drop Unnamed column

In [54]:
df.head()

,Indnum,Group,Activity,Units,Consumption,Quality_of_Life_Importance__1_10,solar_powered_water_heater,gas_water_heater,electric_water_heater_peak_hour,electric_water_heater_off_peak,gas,natural_gas,hybrid,electric_peak_hours,electric_off_peak_hours,jetfuel,TCF,TrQL
0,1,1,Household heating => 70F,hours,2,88,0.0,0.0,0.0,0.0,0.0,0.000436,0,0.0,0.0,0.0,0.000436,176
1,1,1,Household heating < 70F,hours,10,85,0.0,0.0,0.0,0.0,0.0,0.000872,0,0.0,0.0,0.0,0.000872,850
2,1,1,Use of heat pump,hours,0,50,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0
3,1,1,Use of air conditioner,hours,20,45,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,900
4,1,2,shower - short,count,5,98,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,490


In [55]:
indv1_df = df.drop('Group', axis=1).loc[df['Indnum'] == 1]

In [56]:
#indv1_df.head()

In [57]:
gp_model = create_prob('Wells Fargo Challenge', lp.LpMinimize)

## Decision Variables

In [58]:
indv_num = indv1_df.Indnum.unique()
actv_names = indv1_df.Activity.unique()

sources = np.array([
  "solar_powered_water_heater",
  "gas_water_heater",
  "electric_water_heater_peak_hour",
  "electric_water_heater_off_peak",
  "gas",
  "natural_gas",
  "hybrid",
  "electric_peak_hours",
  "electric_off_peak_hours",
  "jetfuel"
])

In [59]:
consumption_indexes = [
    (indv, activity)
    for indv in indv_num
    for activity in actv_names
]

C_ij = lp.LpVariable.dicts('C_ij', consumption_indexes, lowBound=0)

## Objective Function

### $Z_{min} = C_{ij} * \sum CF_{ijk}$

where 
    * i = individual
    * j = activity
    * k = source
    * _C_ is the consumption per unit of an activity

In [62]:
d_vars = []

for indv in indv_num:
    for activity in actv_names:
        total_actv_cf: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'TCF'].values
        d_vars.append(C_ij[(indv, activity)] * head(total_actv_cf))


obj_fn = lp.lpSum(d_vars)

obj_fn

0.000872*C_ij_(1,_'Household_heating_<_70F') + 0.000436*C_ij_(1,_'Household_heating_=__70F') + 0.000354*C_ij_(1,_'car_trips___2__people_with_multiple_end_points') + 0.000554*C_ij_(1,_'car_trips___driver_and_self') + 0.000551*C_ij_(1,_'car_trips__self_only') + 4.2e-05*C_ij_(1,_'wash_up') + 0.0

## Constraints

### $C_{ij} * QL_{ij} >= TrQL_{ij}$

where 
    * i = individual
    * j = activity
    * _QL_ is the quality of life for an activity
    * _TrQL_ is the true quality of life for an activity it is computed as $C_{ij} * QL_{ij}$

In [64]:
tr_ql_conds = []

for indv in indv_num:
    for activity in actv_names:
        ql: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'Quality_of_Life_Importance__1_10'].values
        tr_ql: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'TrQL'].values
        condition = C_ij[(indv, activity)] * head(ql) >= head(tr_ql)
            
        tr_ql_conds.append(condition)

tr_ql_conds

[88*C_ij_(1,_'Household_heating_=__70F') + -176 >= 0,
 85*C_ij_(1,_'Household_heating_<_70F') + -850 >= 0,
 50*C_ij_(1,_'Use_of_heat_pump') + 0 >= 0,
 45*C_ij_(1,_'Use_of_air_conditioner') + -900 >= 0,
 98*C_ij_(1,_'shower___short') + -490 >= 0,
 74*C_ij_(1,_'shower___long_(__3_min)') + -2220 >= 0,
 14*C_ij_(1,_'bath') + 0 >= 0,
 27*C_ij_(1,_'wash_up') + -1188 >= 0,
 20*C_ij_(1,_'use_of_dishwasher') + 0 >= 0,
 28*C_ij_(1,_'use_of_clothes_washer') + -56 >= 0,
 11*C_ij_(1,_'use_of_clothes_dryer') + -33 >= 0,
 0 >= 0,
 35*C_ij_(1,_'use_of__oven') + -315 >= 0,
 15*C_ij_(1,_'use_of_self_clean_feature_of_electric_oven') + -45 >= 0,
 0 >= 0,
 23*C_ij_(1,_'TV_computer_use') + -1472 >= 0,
 42*C_ij_(1,_'air_travel___large_plane') + -115374 >= 0,
 0 >= 0,
 44*C_ij_(1,_'car_trips__self_only') + -19492 >= 0,
 27*C_ij_(1,_'car_trips___driver_and_self') + -1026 >= 0,
 0 >= 0,
 49*C_ij_(1,_'trips_using_public_ground_transportation') + -735 >= 0,
 41*C_ij_(1,_'bags_of_garbage_disposed') + -1066 >= 0,
 

In [65]:
gp_model = add_obj_fn(gp_model, obj_fn)

gp_model = add_constraint(gp_model, tr_ql_conds)

In [66]:
gp_model.solve()

1

In [67]:
lp.LpStatus[gp_model.status]

'Optimal'

In [68]:
lp.value(gp_model.objective)

0.27658499999999997

In [74]:
for indv in indv_num:
    for activity in actv_names:
            ql: np.ndarray = indv1_df.loc[indv1_df['Activity'] == activity, 'Quality_of_Life_Importance__1_10'].values
            print(activity, '| Consumption: ', lp.value(C_ij[(indv, activity)]), '| Quality of Life', head(ql))

Household heating => 70F | Consumption:  2.0 | Quality of Life 88
Household heating < 70F | Consumption:  10.0 | Quality of Life 85
Use of heat pump | Consumption:  0.0 | Quality of Life 50
Use of air conditioner | Consumption:  20.0 | Quality of Life 45
shower - short | Consumption:  5.0 | Quality of Life 98
shower - long (> 3 min) | Consumption:  30.0 | Quality of Life 74
bath | Consumption:  0.0 | Quality of Life 14
wash-up | Consumption:  44.0 | Quality of Life 27
use of dishwasher | Consumption:  0.0 | Quality of Life 20
use of clothes washer | Consumption:  2.0 | Quality of Life 28
use of clothes dryer | Consumption:  3.0 | Quality of Life 11
use of cooking range | Consumption:  None | Quality of Life 0
use of  oven | Consumption:  9.0 | Quality of Life 35
use of self-clean feature of electric oven | Consumption:  3.0 | Quality of Life 15
Small kitchen appliance in the home | Consumption:  None | Quality of Life 0
TV/computer use | Consumption:  64.0 | Quality of Life 23
air trav